In [1]:
import os
import cv2
import glob
import csv

import numpy as np

from math import ceil

In [11]:
IMGFOLDER  = "D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch\\"
PREDFOLDER = "D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\" # Predictions on patches

IMAGEEXT = ".png" # include the '.'
PREDEXT  = ".csv" # include the '.' # I don't think this can be anything other than .csv really...
PREDFNAMESUFFIX = "_boxes" # Between the image name and the extension

#DELIMITER = '_'

CROSS_PATCH_SUPPRESSION = 10 # This is the max number of pixels that if two boxes occur across the patch separator they will be combined

CROP_PADDING = 2 # Border-pad by this many pixels on each detction crop export

DILATE_KERNEL = np.ones((5,2),np.uint8)

In [12]:
patchfolders = [f.path for f in os.scandir(PREDFOLDER) if f.is_dir()]

print(patchfolders)

['D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_369787469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_369788469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_369789469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_370787469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_370788469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_370789469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_371787469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_371788469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annotations\\output_stitch_outputs\\UNet\\03_371789469', 'D:\\tensorflow\\projects\\PlantClumpDetection\\annota

In [4]:
def returnBoxDict(xmin, ymin, xmax, ymax):
    '''
    Construct a box dictionary from xmin, ymin, xmax, and ymax, with (0,0) being the top-left corner
    
    :param int xmin: the location of the min x value of the box
    :param int ymin: the location of the min y value of the box
    :param int xmax: the location of the max x value of the box
    :param int ymax: the location of the max y value of the box
    :return: a dictionary defining a box containing (uint) values for the keys {x,y}{min,max}
    :rtype: dict
    '''
    tempDict = {}
    tempDict["xmin"] = xmin
    tempDict["ymin"] = ymin
    tempDict["xmax"] = xmax
    tempDict["ymax"] = ymax
    return tempDict

In [5]:
def decodeBoxFromDict(box):
    '''
    Get xmin, ymin, xmax, and ymax from a box dictionary
    
    :param dict box: a dictionary defining a box containing (uint) values for the keys {x,y}{min,max}
    :return: a tuple containing (uint) xmin, ymin, xmax, ymax from the appropriate keys in box
    :rtype: tup -> int, int, int, int
    '''
    xmin = box["xmin"]
    ymin = box["ymin"]
    xmax = box["xmax"]
    ymax = box["ymax"]
    return xmin, ymin, xmax, ymax

In [6]:
def combineBoxesAtBoundaries(boxes, boundaries, pxthresh, sort_key="xmin", no_y_check=False):
    '''
    Combines boxes within a list of boxes if they are within pxthresh on either side of a boundary in boundaries
    
    :param list boxes: a list of box dictionaries
    :param list boundaries: a list of integer boundaries
    :param int pxthresh: an integer threshold
    :return: a list of boxes where any box combinations at boundaries have occurred 
    :rtype: list
    '''
    pxthresh += 1 # In case of pesky off by one errors with < and >
    
    xminSorted = sorted(boxes, key=lambda x: x["xmin"])
    xmaxSorted = sorted(boxes, key=lambda x: x["xmax"])
    #print(xminSorted) # DEBUG
    #print(xmaxSorted) # DEBUG
    
    toCombine     = {}
    toCombineFlat = []
    xminIdx       = 0
    xmaxIdx       = 0
    
    for boundary in boundaries:
        while xmaxIdx < len(xmaxSorted):
            xmaxCurr    = xmaxSorted[xmaxIdx]
            xmaxCurrVal = xmaxCurr["xmax"]
            xmaxIdx += 1
            if xmaxCurrVal > boundary:
                break
            elif boundary-xmaxCurrVal < pxthresh:
                while xminIdx < len(xminSorted):
                    xminCurr    = xminSorted[xminIdx]
                    xminCurrVal = xminCurr["xmin"]
                    xminIdx += 1
                    if xminCurr is xmaxCurr:
                        continue
                    elif 0 <= xminCurrVal-boundary < pxthresh:
                        try:
                            if xmaxCurr not in toCombine[str(boundary)]:
                                toCombine[str(boundary)].append(xmaxCurr)
                                toCombineFlat.append(xmaxCurr)
                            if xminCurr not in toCombine[str(boundary)]:
                                toCombine[str(boundary)].append(xminCurr)
                                toCombineFlat.append(xminCurr)
                                print("COMBINING@boundary={} (again)".format(boundary))
                        except KeyError:
                            print("COMBINING@boundary={}".format(boundary))
                            toCombine[str(boundary)] = [xmaxCurr, xminCurr]
                            toCombineFlat.append(xmaxCurr)
                            toCombineFlat.append(xminCurr)
                    elif xminCurrVal <= boundary:
                        continue
                    else:
                        break
            else:
                continue
    
    toReturn   = []
    
    for box in boxes:
        if box not in toCombineFlat:
            toReturn.append(box)
    
    for boundary, comboList in toCombine.items():
        yminList = [box["ymin"] for box in comboList]
        ymaxList = [box["ymax"] for box in comboList]
        if (no_y_check) or (abs(np.mean(yminList)-min(yminList)) < pxthresh) or (abs(np.mean(ymaxList)-min(ymaxList)) < pxthresh):
            toReturn.append(returnBoxDict(min([box["xmin"] for box in comboList]), 
                                          min(yminList),
                                          max([box["xmax"] for box in comboList]),
                                          max(ymaxList)
                                         ))
        else:
            toReturn.extend(combo)
    
    toReturn.sort(key=lambda x: x[sort_key])
    return toReturn

# TESTS #
boxesTest = []
boxesTest.append(returnBoxDict(1,  5, 12, 10))
boxesTest.append(returnBoxDict(18, 4, 21, 11))
boxesTest.append(returnBoxDict(29, 7, 40, 12))
boxesTest.append(returnBoxDict(48, 5, 60, 10))
boxesTest.append(returnBoxDict(70, 6, 85, 10))
boxesTest.append(returnBoxDict(85, 4, 90, 9))
boxesTest.append(returnBoxDict(94, 4, 99, 9))
boxesTest.append(returnBoxDict(101, 3, 110, 12))
boxesTest.append(returnBoxDict(103, 1, 111, 9))

boxesOut = combineBoxesAtBoundaries(boxesTest, [25, 85, 100], 4)
for box in boxesOut:
    print(box)

COMBINING@boundary=25
COMBINING@boundary=85
COMBINING@boundary=100
COMBINING@boundary=100 (again)
{'xmin': 1, 'ymin': 5, 'xmax': 12, 'ymax': 10}
{'xmin': 18, 'ymin': 4, 'xmax': 40, 'ymax': 12}
{'xmin': 48, 'ymin': 5, 'xmax': 60, 'ymax': 10}
{'xmin': 70, 'ymin': 4, 'xmax': 90, 'ymax': 10}
{'xmin': 94, 'ymin': 1, 'xmax': 111, 'ymax': 12}


In [7]:
'''toRemove = []
toReturn = []

for boxA in boxes:
    comboList = []
    for boxB in boxes:
        if boxA is not boxB:
            if (boxA["xmin"] > boxB["xmax"] or boxB["xmin"] > boxA["xmax"] or boxA["ymin"] > boxB["ymax"] or boxB["ymin"] > boxA["ymax"]):
                pass
            else:
                comboList.append(boxB)
    if not comboList'''

'toRemove = []\ntoReturn = []\n\nfor boxA in boxes:\n    comboList = []\n    for boxB in boxes:\n        if boxA is not boxB:\n            if (boxA["xmin"] > boxB["xmax"] or boxB["xmin"] > boxA["xmax"] or boxA["ymin"] > boxB["ymax"] or boxB["ymin"] > boxA["ymax"]):\n                pass\n            else:\n                comboList.append(boxB)\n    if not comboList'

In [13]:
for patchfolder in patchfolders:
    
    patchfoldername     = os.path.splitext(os.path.basename(patchfolder))[0]
    pathimagename       = patchfoldername + IMAGEEXT
    boxesCSVfilename    = patchfoldername + PREDFNAMESUFFIX + PREDEXT
    boxesCSVfilenameOut = patchfoldername + "_crops" + PREDEXT
    print("***** PROCESSING {} ... *****".format(pathimagename)) # DEBUG
    
    # Boxes
    boxes = []
    
    with open(os.path.join(patchfolder, boxesCSVfilename), newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        skipFirst = False
        for row in reader:
            if not skipFirst: # To remove the first line contianing only the headers 
                skipFirst = True
                continue
            boxes.append(returnBoxDict(int(row[0]), int(row[1]), int(row[2]), int(row[3])))
    
    #print(boxes) # DEBUG
    
    frame = cv2.imread(os.path.join(IMGFOLDER, pathimagename))
    if frame is None:
        print("WARNING: {} cannot be read. Skipping...".format(pathimagename))
        continue
    
    h, w, c = frame.shape
    numPatches    = int(ceil(w/h)) # Ceil 'cause we don't want to discard last information
    boundaryArr   = list(range(h,numPatches*h,h))
    #print(boundaryArr) # DEBUG
    
    new = frame.copy()
    blank = np.zeros((h, w, 3)).astype('uint8')
    #print(blank.shape) # DEBUG
    
    boxesOut = combineBoxesAtBoundaries(boxes, boundaryArr, CROSS_PATCH_SUPPRESSION, no_y_check=True)
    #print(boxesOut) # DEBUG
    
    padding = 0#CROSS_PATCH_SUPPRESSION//2
    for box in boxesOut:
        xmin, ymin, xmax, ymax = decodeBoxFromDict(box)
        cv2.rectangle(blank, (xmin-padding, ymin-padding), (xmax+padding, ymax+padding), (255, 255, 255), thickness=cv2.FILLED)
    
    mask = cv2.dilate(blank, DILATE_KERNEL, iterations=1)
    
    maskImageName = patchfoldername+"_mask"+IMAGEEXT
    cv2.imwrite(os.path.join(patchfolder, maskImageName), mask) # Export detection mask
    
    boxesOut2 = []
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    im2, contours, hierarchy = cv2.findContours(gray_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        boxesOut2.append(returnBoxDict(x, y, x+w, y+h))
    
    boxesOut2 = sorted(boxesOut2, key=lambda x: x["xmin"])
    
    # Draw detection rectangles and save crops
    with open(os.path.join(patchfolder, boxesCSVfilenameOut), 'w', newline='') as csvfile:
        label = 1
        csvwriter = csv.writer(csvfile, delimiter=',',
                               quotechar='|', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['label', 'xmin', 'ymin', 'xmax', 'ymax'])
        
        for box in boxesOut2:
            xmin, ymin, xmax, ymax = decodeBoxFromDict(box)
            cv2.rectangle(new, (xmin, ymin), (xmax, ymax), (0, 255, 0), thickness=2)
            cropImageName = patchfoldername+"_"+'%03d'%label+IMAGEEXT
            cv2.imwrite(os.path.join(patchfolder, cropImageName), frame[ymin-CROP_PADDING:ymax+CROP_PADDING,xmin-CROP_PADDING:xmax+CROP_PADDING,:])
            csvwriter.writerow(['%03d'%label, xmin, ymin, xmax, ymax])
            label += 1
    
    cv2.imwrite(os.path.join(patchfolder, pathimagename), new) # Export detection image
    
    #break # DEBUG

***** PROCESSING 03_369787469.png ... *****
***** PROCESSING 03_369788469.png ... *****
***** PROCESSING 03_369789469.png ... *****
***** PROCESSING 03_370787469.png ... *****
***** PROCESSING 03_370788469.png ... *****
***** PROCESSING 03_370789469.png ... *****
***** PROCESSING 03_371787469.png ... *****
***** PROCESSING 03_371788469.png ... *****
***** PROCESSING 03_371789469.png ... *****
***** PROCESSING 05_375793474.png ... *****
COMBINING@boundary=4972
***** PROCESSING 05_375793475.png ... *****
COMBINING@boundary=2700
COMBINING@boundary=4950
***** PROCESSING 05_375793476.png ... *****
COMBINING@boundary=3178
COMBINING@boundary=4994
***** PROCESSING 05_375794474.png ... *****
COMBINING@boundary=4580
***** PROCESSING 05_375794475.png ... *****
COMBINING@boundary=2736
COMBINING@boundary=4560
COMBINING@boundary=5016
***** PROCESSING 05_375794476.png ... *****
COMBINING@boundary=3248
***** PROCESSING 05_375795475.png ... *****
COMBINING@boundary=2280
COMBINING@boundary=2736
***** PR

***** PROCESSING 09_387805486.png ... *****
COMBINING@boundary=2400
COMBINING@boundary=2880
COMBINING@boundary=3360
COMBINING@boundary=3840
COMBINING@boundary=4320
COMBINING@boundary=4800
***** PROCESSING 09_387805487.png ... *****
COMBINING@boundary=2420
COMBINING@boundary=2904
***** PROCESSING 09_387806485.png ... *****
COMBINING@boundary=2375
COMBINING@boundary=3325
***** PROCESSING 09_388806486.png ... *****
COMBINING@boundary=2405
COMBINING@boundary=3367
COMBINING@boundary=4329
COMBINING@boundary=4810
***** PROCESSING 09_388806487.png ... *****
COMBINING@boundary=2415
COMBINING@boundary=2898
COMBINING@boundary=3381
COMBINING@boundary=3864
***** PROCESSING 10_388807487.png ... *****
COMBINING@boundary=1900
COMBINING@boundary=3800
COMBINING@boundary=4275
COMBINING@boundary=4275 (again)
COMBINING@boundary=4750
***** PROCESSING 10_388807488.png ... *****
COMBINING@boundary=1924
COMBINING@boundary=1924 (again)
COMBINING@boundary=2405
COMBINING@boundary=3848
COMBINING@boundary=4329
COMB

***** PROCESSING 17_402822501.png ... *****
COMBINING@boundary=926
COMBINING@boundary=2315
COMBINING@boundary=2778
COMBINING@boundary=4630
***** PROCESSING 17_402822502.png ... *****
COMBINING@boundary=2778
***** PROCESSING 17_402822503.png ... *****
COMBINING@boundary=2315
COMBINING@boundary=2778
COMBINING@boundary=4630
***** PROCESSING 17_403821501.png ... *****
COMBINING@boundary=2305
***** PROCESSING 17_403822501.png ... *****
COMBINING@boundary=2355
COMBINING@boundary=4239
***** PROCESSING 17_403822502.png ... *****
COMBINING@boundary=6097
***** PROCESSING 17_403822503.png ... *****
COMBINING@boundary=2350
COMBINING@boundary=4700
***** PROCESSING 17_403823502.png ... *****
COMBINING@boundary=4257
***** PROCESSING 17_403823503.png ... *****
COMBINING@boundary=1904
COMBINING@boundary=2380
COMBINING@boundary=4760
***** PROCESSING 17_404821501.png ... *****
COMBINING@boundary=1395
COMBINING@boundary=2325
***** PROCESSING 17_404821502.png ... *****
COMBINING@boundary=2315
***** PROCESS

***** PROCESSING 24_417836518.png ... *****
COMBINING@boundary=1470
COMBINING@boundary=1960
COMBINING@boundary=2940
COMBINING@boundary=3430
COMBINING@boundary=4410
***** PROCESSING 24_417837517.png ... *****
COMBINING@boundary=2385
COMBINING@boundary=2862
COMBINING@boundary=4293
COMBINING@boundary=4770
COMBINING@boundary=5247
***** PROCESSING 24_417837518.png ... *****
COMBINING@boundary=1449
COMBINING@boundary=2898
COMBINING@boundary=3864
COMBINING@boundary=4347
***** PROCESSING 24_418836516.png ... *****
COMBINING@boundary=2355
COMBINING@boundary=2826
COMBINING@boundary=4710
COMBINING@boundary=5181
COMBINING@boundary=6123
***** PROCESSING 24_418836517.png ... *****
COMBINING@boundary=1461
COMBINING@boundary=2922
COMBINING@boundary=3896
***** PROCESSING 24_418836518.png ... *****
COMBINING@boundary=1446
COMBINING@boundary=2892
***** PROCESSING 24_418837517.png ... *****
COMBINING@boundary=1440
COMBINING@boundary=2880
COMBINING@boundary=3840
COMBINING@boundary=4320
***** PROCESSING 24_

***** PROCESSING 29_427847528.png ... *****
COMBINING@boundary=1467
COMBINING@boundary=3912
***** PROCESSING 29_428845526.png ... *****
COMBINING@boundary=1446
COMBINING@boundary=2410
COMBINING@boundary=3856
***** PROCESSING 29_428846526.png ... *****
COMBINING@boundary=1467
COMBINING@boundary=2445
COMBINING@boundary=2445 (again)
COMBINING@boundary=3912
***** PROCESSING 29_428846527.png ... *****
COMBINING@boundary=1449
COMBINING@boundary=2415
COMBINING@boundary=3864
***** PROCESSING 29_428846528.png ... *****
COMBINING@boundary=1449
COMBINING@boundary=2415
COMBINING@boundary=3864
COMBINING@boundary=5313
***** PROCESSING 29_428847526.png ... *****
COMBINING@boundary=1455
COMBINING@boundary=3880
COMBINING@boundary=4850
***** PROCESSING 29_428847528.png ... *****
COMBINING@boundary=1467
COMBINING@boundary=2934
***** PROCESSING 29_429846528.png ... *****
COMBINING@boundary=1431
COMBINING@boundary=2385
COMBINING@boundary=2385 (again)
COMBINING@boundary=3816
***** PROCESSING 29_429847526.pn

***** PROCESSING 36_442860541.png ... *****
***** PROCESSING 36_442860542.png ... *****
COMBINING@boundary=2850
***** PROCESSING 36_442861541.png ... *****
***** PROCESSING 36_442861542.png ... *****
***** PROCESSING 36_443861541.png ... *****
***** PROCESSING 36_443861542.png ... *****
COMBINING@boundary=2850
***** PROCESSING 37_442861541.png ... *****
COMBINING@boundary=1864
COMBINING@boundary=3728
COMBINING@boundary=5126
***** PROCESSING 37_442861542.png ... *****
COMBINING@boundary=1880
COMBINING@boundary=3760
COMBINING@boundary=4700
***** PROCESSING 37_442861543.png ... *****
COMBINING@boundary=3744
COMBINING@boundary=5148
***** PROCESSING 37_442862541.png ... *****
COMBINING@boundary=1872
COMBINING@boundary=3744
***** PROCESSING 37_442862542.png ... *****
COMBINING@boundary=3374
COMBINING@boundary=4338
COMBINING@boundary=4820
***** PROCESSING 37_442862543.png ... *****
COMBINING@boundary=3304
COMBINING@boundary=3776
***** PROCESSING 37_443861541.png ... *****
COMBINING@boundary=1

***** PROCESSING 44_454873554.png ... *****
***** PROCESSING 44_455873553.png ... *****
COMBINING@boundary=1900
***** PROCESSING 44_455873554.png ... *****
COMBINING@boundary=1449
COMBINING@boundary=4830
